# SQL Tables

> Class declarations of the database tables.

In [ ]:
#| default_exp tables

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| exporti

from sqlalchemy import Column, Integer, String, Enum, Boolean, PickleType, ForeignKey, Table, Float
from sqlalchemy.ext.declarative import declarative_base

In [ ]:
#|exporti
Base = declarative_base()

In [ ]:
#| exports

class Image(Base):
    "Image SQL class"
    __tablename__ = "image"
    image_id = Column(Integer, primary_key=True) # Image primary key
    path = Column(String) # Absolute or relative path
    identity = Column(String) # Person identity of the image
    source = Column(String) # Database the image belongs to.

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()